# Up-the-ramp Sampling with calwf3

This training is meant to familiarize you with the behavior of IR detectors and the basic corrections that are needed for scientific observations. In this training, you will have an introduction to the pipeline reductions of the HST instrument WFC3 also known as WF3. Helped by Bryan Hilbert and Megan Sosey, I (Massimo Robberto) have put together this IPython notebook to "x-ray" the 11 steps performed by CALWF3-IR to extract a count-rate image from a raw ramp. (For comments, suggestions and to share improved versions, e-mail robberto@stsci.edu)

To begin, copy over the following directory of example data to your central store location:

    cp -rfv /grp/hst/riab/training/up_the_ramp_sampling/WFC3_tutorial /user/$USER/WFC3_tutorial

Change directories to the new `WFC3_tutorial` folder you just created:

    cd /user/$USER/WFC3_tutorial/main/

This document is written as a Jupyter notebook to run on your own computer. You will need to obtain the notebook named `up_the_ramp_calwf3` from this website with the following command:

    curl -L https://stsci-riab.github.io/riatraining/_sources/up_the_ramp_calwf3.txt > up_the_ramp_calwf3.ipynb
    
Start the notebook server:

    jupyter notebook

and load `up_the_ramp_calwf3.ipynb`, i.e. your copy of this document. The exercise runs on an arbitrary image with the associated calibration files; any other image + calibration file downloaded from the archive should work.

Run through the code cells in the rest of this document, then try the exercise in the box immediately below.

<div class="alert alert-info">

**Exercise:** Plot counts vs. total integration time from the IMA file for the pixel in the tutorial. Describe the behavior of the signal. How is the pixel used for the final slope calculation?

</div>

<div class="alert alert-info">

**Tip:** Use header keyword 'SAMPTIME' from each 'SCI' extension of the IMA file. Make sure to check the 'BUNIT' keyword.

</div>

In [2]:
%matplotlib inline
import os
from astropy.io import fits
import numpy as np

## Load and check the raw data file

In [3]:
filebase = 'icol26k3q'
file_raw = filebase+'_raw.fits' 
hdulist=fits.open(file_raw)
hdulist.info()

Filename: icol26k3q_raw.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU     204   ()              
1    SCI         ImageHDU        83   (1024, 1024)   int16 (rescales to uint16)   
2    ERR         ImageHDU        44   ()              
3    DQ          ImageHDU        36   ()              
4    SAMP        ImageHDU        30   ()              
5    TIME        ImageHDU        30   ()              
6    SCI         ImageHDU        83   (1024, 1024)   int16 (rescales to uint16)   
7    ERR         ImageHDU        44   ()              
8    DQ          ImageHDU        36   ()              
9    SAMP        ImageHDU        30   ()              
10   TIME        ImageHDU        30   ()              
11   SCI         ImageHDU        83   (1024, 1024)   int16 (rescales to uint16)   
12   ERR         ImageHDU        44   ()              
13   DQ          ImageHDU        36   ()              
14   SAMP        ImageHDU        30   ()              
15  

## Unpack the datacube

In this excercise I care about the counts, the error and the data quality 

In [4]:
cube = np.zeros((5,1024,1024))
error =  np.zeros((5,1024,1024))  #adding error matrix
dq =  np.zeros((5,1024,1024))  #DQ matrix
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data

## Pick up  a pixel to be followed to the end

In [5]:
#Good pixels to follow are the weird one, e.g. the those at the peak of a PSF (saturated), CRs, hot pixels etc/
ii=441+5  
jj=716+5

#If you pick up a pixel inspecting an image with xv, remember that python (like IDL) starts from 0,0, while xv starts from 1,1
#There are also the 5 rows/columns of reference pixels, that will be trimmed out at the end, i.e. the final data product 
#is a 1014x1014 image, not a 1024x1024. 
#I enter the coordinates after inspecting the final 1014x1014 image, so I add +5

In [6]:
#check what we have as baseline
print(cube[:,jj,ii])
print(error[:,ii,jj])
print(dq[:,ii,jj])

#and remember that WFC3 stored the ramp in reverse order last->first. The first counts are higher than the last.

[ 54657.  54631.  54790.  38868.  25537.]
[ nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan]


In [7]:
#Prepare an array storing the results, step by step, for a final plot
listout = np.zeros((13,5))
listout[0,:] = cube[:,jj,ii]
errout = np.zeros((13,5))
errout[0,:] = error[:,jj,ii]
dqout = np.zeros((13,5))
dqout[0,:] = error[:,jj,ii]
stepname = ['RAW']   #this is the flag for the original data.

In [8]:
#setup and cleanp
os.putenv("iref","../calfiles/")
if os.path.exists('./'+filebase+"_flt.fits"):
    os.remove('./'+filebase+"_flt.fits")
if os.path.exists('./'+filebase+"_ima.fits"):
    os.remove('./'+filebase+"_ima.fits")

In [9]:
from wfc3tools import calwf3
from wfc3tools import wf3ir

def calwf3_run(file, stepdict, kw=None, value=True):
    '''
    Run only some of the calwf3 steps.
    stepdict is a dictionary where the keys are the names of the calwf3 steps,
    and the values are True or False
    '''

    #update the dict if kw isn't NONE
    if kw:
        stepdict[kw]=value
    # make a backup copy of the original file
    os.system("cp "+ file+" backup_copy_"+file)

    #declare which steps are going to be run
    hdulist = fits.open(file, mode='update')

    #reset all steps to OMIT, then set only those
    #requested to PERFORM
    hdulist[0].header['DQICORR'] = 'OMIT'
    hdulist[0].header['ZSIGCORR'] = 'OMIT'
    hdulist[0].header['ZOFFCORR'] = 'OMIT'
    hdulist[0].header['DARKCORR'] = 'OMIT'
    hdulist[0].header['BLEVCORR'] = 'OMIT'
    hdulist[0].header['NLINCORR'] = 'OMIT'
    hdulist[0].header['FLATCORR'] = 'OMIT'
    hdulist[0].header['DRIZCORR'] = 'OMIT'
    hdulist[0].header['PHOTCORR'] = 'OMIT'
    hdulist[0].header['UNITCORR'] = 'OMIT'
    hdulist[0].header['CRCORR'] = 'OMIT'


    if stepdict['dqi']:
        hdulist[0].header['DQICORR'] = 'PERFORM'

    if stepdict['zsig']:
        hdulist[0].header['ZSIGCORR'] = 'PERFORM'

    if stepdict['zoff']:
        hdulist[0].header['ZOFFCORR'] = 'PERFORM'

    if stepdict['dark']:
        hdulist[0].header['DARKCORR'] = 'PERFORM'

    if stepdict['blev']:
        hdulist[0].header['BLEVCORR'] = 'PERFORM'

    if stepdict['nlin']:
        hdulist[0].header['NLINCORR'] = 'PERFORM'

    if stepdict['flat']:
        hdulist[0].header['FLATCORR'] = 'PERFORM'

    if stepdict['driz']:
        hdulist[0].header['DRIZCORR'] = 'PERFORM'

    if stepdict['phot']:
        hdulist[0].header['PHOTCORR'] = 'PERFORM'

    if stepdict['unit']:
        hdulist[0].header['UNITCORR'] = 'PERFORM'

    if stepdict['cr']:
        hdulist[0].header['CRCORR'] = 'PERFORM'

    hdulist.flush()

    #run calwf3
    calwf3(file)

    imafile = file[0:10] + 'ima.fits'

    print('calwf3 complete: output saved in %s'%imafile)
    return imafile

The following tasks in the wfc3tools package can be run with TEAL:
  calwf3    pstack    pstat     wf32d     wf3ccd    wf3cte    wf3ir     wf3rej


## Step 1: DQI – IR Data Quality Array Initialization

- Header Switch: DQICORR

- Header Keywords Updated: None

- Reference Files: BPIXTAB (*_bpx.fits)

DQICORR populates the data quality (DQ) array in all IR readouts by reading a table of known bad pixels for the detector, stored in the ‘Bad Pixel’ reference table (BPIXTAB). The types of bad pixels that can be flagged are listed in Table 2.5.
The DQ array may already have been populated with some values to flag pixels that were affected by telemetry problems during downlink. Other DQ values will only be marked during further processing (such as cosmic-ray rejection).
The reference file for data quality initialization, BPIXTAB, is selected based on the value of the DETECTOR keyword only.

In [10]:
# We put the steps in a dictionary, flagging as True those that we want to run. 
stepdict = {'dqi':True,'zsig':False,'zoff':None,'dark':None,'blev':None,'nlin':None,'flat':None,'driz':None,'phot':None,'unit':None,'cr':None}
#and GO
calwf3_run(file_raw, stepdict, kw=None, value=False)
#
#Read and print the results of this first calibration step
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
#
#store in the summary cubes
listout[1,:] = cube[:,jj,ii]
errout[1,:] = error[:,jj,ii]
dqout[1,:] = dq[:,jj,ii]
stepname.append('DQI')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 54657.  54631.  54790.  38868.  25537.]
[ 153.07583618  153.03955078  153.26138306  129.16957092  104.82255554]
[ 0.  0.  0.  0.  0.]


## Step 2: ZSIG – IR Zero-Read Signal Correction

- Header Switch: ZSIGCORR

- Header Keywords Updated: None

- Reference Files: DARKFILE (\*_drk.fits), NLINFILE (\*_lin.fits)

At the beginning of an IR observation the detector pixels are reset to the bias level and then read out to record that bias level. An interval of approximately 2.9 seconds elapses between the time each pixel is reset and then read. Because the IR channel does not have a shutter, signal from external sources starts to accumulate during that 2.9 second interval. When the initial (or ‘zeroth’) read is later subtracted from subsequent readouts, any signal in the zeroth read will also be subtracted. Because linearity correction and saturation checking (described below) both depend on the absolute signal level in a pixel at the time it was read, the signal in the zeroth read from bright sources can be large enough to lead to inaccurate linearity corrections, as well as the failure to detect saturation conditions, in the NLINCORR calibration step.

The ZSIGCORR step is used to estimate the amount of source signal in the zeroth read and to supply this estimate to the NLINCORR step for linearity corrections and saturation checking. ZSIGCORR estimates the signal in the zeroth read by first measuring the signal in each pixel between the zeroth and first reads, and then scaling that signal to the effective exposure time of the zeroth read (nominally 2.9 seconds). Pixels that have an estimated zeroth read signal greater than 5 times their estimated uncertainty (noise) value are assumed to contain detectable signal; those below this threshold are ignored. The estimated zeroth read signal is then passed, as an in-memory image, to the NLINCORR step, which accounts for that signal when applying linearity corrections and saturation checking on the zeroth-read subtracted images.

Note that this technique will not work well for pixels covered by targets that are so bright that the signal is already beginning to saturate in either the zeroth or first readouts, because then it is difficult to accurately estimate the zeroth-read signal. ZSIGCORR therefore checks for saturation in the zeroth and first read images and flags those pixels.

Pixels that are determined to have detectable signal in the zeroth read are flagged in the DQ arrays of the output ima file with a data quality value of 2048.
The reference files used in this step, DARKFILE and NLINFILE, are selected from CDBS based on the values of the DETECTOR, CCDAMP, CCDGAIN, SAMP_SEQ, and SUBTYPE keywords. The DARKFILE file is used to subtract dark current from the first-minus-zero read difference image before using it to estimate incoming signal levels and the NLINFILE is used to perform saturation checking.


In [11]:
#Everything now repeats the same pattern, adding steps to the dictionary
#
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':None,'dark':None,'blev':None,'nlin':None,'flat':None,'driz':None,'phot':None,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[2,:] = cube[:,jj,ii]
errout[2,:] = error[:,jj,ii]
dqout[2,:] = dq[:,jj,ii]
stepname.append('ZSIG')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 54657.  54631.  54790.  38868.  40529.]
[ 153.07583618  153.03955078  153.26138306  129.16957092  131.88861084]
[    0.     0.     0.     0.  2048.]


By the way, since we start using them, let's check the values stored in the reference files for our pixel

In [12]:
nlinfile = hdulist[0].header['NLINFILE']
iref = "../calfiles/"
hdulist_nlinfile=fits.open(iref+nlinfile[5:])  #not too elegant, but it works
#
#the "superbias" value is at extension 17
bias = hdulist_nlinfile[17].data
print(cube[:,jj,ii])
#
#the "saturation" value is at extension 16
saturation = hdulist_nlinfile[16].data
print(saturation[jj,ii])
#
#the "readnoise" value is at extension 18
ron = hdulist_nlinfile[18].data
print(ron[jj,ii])




[ 54657.  54631.  54790.  38868.  40529.]
33956.9314645
29.2343


In [13]:
#there is something strange in these values: the saturation value is way too low with respect 
#to the actual values at the peak of a bright star.
#
#However, if we subtract the bias from the data...
print(cube[:,jj,ii]-bias[jj,ii])
#and add the bias at the saturation value...
print(saturation[jj,ii]+bias[jj,ii])
#we are basically ok. Not transparent, but the pipeline is a black box that takes care of this...

[ 44112.  44086.  44245.  28323.  29984.]
44501.9314645


## Step 3: ZOFF – IR Zero-read Image Subtraction

- Header Switch: ZOFFCORR

- Header Keywords Updated: None

- Reference Files: None

ZOFFCORR subtracts the zeroth read from all readouts in the exposure, including the zeroth read itself, resulting in a zero-read image that is exactly zero in the remainder of processing. The zeroth-read image is propagated through the remaining processing steps and included in the output products, so that a complete history of error estimates and data quality (DQ) flags is preserved.


In [14]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':None,'blev':None,'nlin':None,'flat':None,'driz':None,'phot':None,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[3,:] = cube[:,jj,ii]
errout[3,:] = error[:,jj,ii]
dqout[3,:] = dq[:,jj,ii]
stepname.append('ZOFF')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 29120.  29094.  29253.  13331.  14992.]
[ 111.88817596  111.83851624  112.14187622   75.97058105   80.50688171]
[    0.     0.     0.     0.  2048.]


## Step 4: DARK – IR Dark Image Subtraction

- Header Switch: DARKCORR

- Header Keywords Updated: MEANDARK

- Reference File: DARKFILE (*_drk.fits)

DARKCORR subtracts the detector dark current from the science data. Due to potential non-linearities in some of the signal components, such as reset-related effects in the first one or two reads of an exposure, the dark current subtraction is not applied by simply scaling a generic reference dark image to the exposure time and then subtracting it. Instead, a library of dark current images is maintained that includes darks taken in each of the available predefined MultiAccum sample sequences, as well as the available sub-array readout modes. The MultiAccum dark reference file is subtracted read-by-read from the stack of science image readouts. Thus there is an exact match in the timings and other characteristics of the dark image that is subtracted from each science readout.
The DARKFILE reference file must have the same values for the DETECTOR, CCDAMP, CCDGAIN, SAMP_SEQ, and SUBTYPE keywords as the science image.

In [15]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':None,'nlin':None,'flat':None,'driz':None,'phot':None,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[4,:] = cube[:,jj,ii]
errout[4,:] = error[:,jj,ii]
dqout[4,:] = dq[:,jj,ii]
stepname.append('DARK')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 29115.75585938  29093.2890625   29254.41796875  13331.67675781  14992.        ]
[ 111.89044189  111.84100342  112.14394379   75.97402954   80.50688171]
[    0.     0.     0.     0.  2048.]


## Step 5: BLEV – IR Bias Level Correction From Reference Pixels

- Header Switch: BLEVCORR

- Header Keywords Updated: MEANBLEV

- Reference Files: OSCNTAB (*_osc.fits)

BLEVCORR uses the reference pixels located around the perimeter of the IR detector to track and remove changes in the bias level that occur during an exposure. For each raw readout, the mean signal level of the reference pixels is computed and subtracted from the image, and recorded in the MEANBLEV keyword in the SCI header of each readout.
The reference pixels located at the ends of each image row are used in this computation. Reference pixels are also located along the bottom and top of the image, but these have been found to be less reliable and are not used. As with the UVIS overscan correction, the boundaries of the reference pixel regions that are used in the computation are defined in the OSCNTAB reference table, in the BIASSECT* columns. The BIASSECTA[1,2] values indicate the starting and ending column numbers for the reference pixels on the left edge of the image, and the BIASSECTB[1,2] give the values for the right side of the image.
The reference pixel regions are maintained throughout the remainder of processing, but are usually ignored or skipped over in the actual application of calibration algorithms. They are left in place in the calibrated data stored in the ima file at the end of processing, but are trimmed from the flt image file.
The reference file for bias level correction, OSCNTAB, is selected from CDBS based on the value of the DETECTOR keyword only.


In [16]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':True,'nlin':None,'flat':None,'driz':None,'phot':None,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[5,:] = cube[:,jj,ii]
errout[5,:] = error[:,jj,ii]
dqout[5,:] = dq[:,jj,ii]
stepname.append('BLEV')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 29119.53320312  29096.16796875  29255.99609375  13332.16015625
  14991.99902344]
[ 111.8976593   111.84650421  112.1469574    75.97538757   80.50688171]
[    0.     0.     0.     0.  2048.]


## Step 6: NLIN – IR Non-Linearity Correction

- Header Switch: NLINCORR

- Header Keywords Updated: None

- Reference File: NLINFILE (*_lin.fits)

NLINCORR corrects the integrated counts in the science images for the non-linear response of the detector and flags pixels that go into saturation. The observed response of the detector can be conveniently represented by two regimes:

1. At low and intermediate signal levels the detector response deviates from the incident flux in a way that is correctable using the following expression:
where c1, c2. c 3 and c4 are the correction coefficients, F is the uncorrected flux (in DN) and Fc is the corrected flux. The current form of the correction uses a third-order polynomial, as shown here, but the algorithm can handle an arbitrary number of coefficients. The number of coefficients and error terms are given by the values of the NCOEF and NERR keywords in the header of the NLINFILE.

2. At high signal levels as saturation sets in the response becomes highly non-linear and is not correctable to a scientifically useful degree.
This step uses the NLINFILE reference file, which includes a set of images containing the cn correction coefficients and their variances at each pixel. The [NODE,1] image extension in the NLINFILE gives the saturation value for each pixel, in units of DN. Each pixel that has an input value below its defined saturation level is corrected according to the equation above. Pixels at or above their saturation values receive no correction and are flagged as saturated in the DQ array for the readout. Any pixel flagged as saturated in a given readout is also automatically flagged as saturated in all subsequent readouts.
As mentioned in the description of the ZSIGCORR routine, the estimated amount of signal in the zeroth read of the exposure is temporarily added back into the signal of each pixel during the NLINCORR step, before the pixel is checked for saturation or receives the linearity correction. Once the correction has been applied, the zero read signal is again removed. This process only occurs if the ZSIGCORR step is turned on during processing.
The NLINFILE reference files is selected based on the value of the DETECTOR keyword only.


In [17]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':True,'nlin':True,'flat':None,'driz':None,'phot':None,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)

hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[6,:] = cube[:,jj,ii]
errout[6,:] = error[:,jj,ii]
dqout[6,:] = dq[:,jj,ii]
stepname.append('NLIN')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 29119.53320312  29096.16796875  29255.99609375  13980.02246094
  15081.25097656]
[ 111.8976593   111.84650421  112.1469574    75.97538757   80.50688171]
[ 2304.  2304.  2304.  2048.  2048.]


## Step 7: FLAT – IR Flat-field Image Correction

- Header Switch: FLATCORR

- Header Keywords Updated: None

- Reference Files:
PFLTFILE(*_pfl.fits),LFLTFILE(*_lfl.fits),DFLTFILE
(*_dfl.fits)

FLATCORR corrects for pixel-to-pixel and large-scale sensitivity variations across the detector by dividing the science images by one or more flat-field images. A combined flat is created within calwf3 using up to three flat-field reference files: the pixel-to-pixel flat (c), the low-order flat (LFLTFILE), and the delta flat (DFLTFILE). FLATCORR also multiplies the science data by the detector gain so that the calibrated data will be in units of electrons per second (or electrons if UNITCORR is not performed).

The PFLTFILE is a pixel-to-pixel flat-field correction file containing the small-scale flat-field variations. The PFLTFILE is always used in the calibration pipeline, while the other two flats are optional. The LFLTFILE is a low-order flat that corrects for any large-scale sensitivity variations across the detector. This file can be stored as a binned image, which is then expanded when being applied by calwf3. Finally, the DFLTFILE is a delta-flat containing any needed changes to the small-scale PFLTFILE.

If the LFLTFILE and DFLTFILE are not specified in the SCI header, only the PFLTFILE is used for the flat-field correction. If two or more reference files are specified, they are read in and multiplied together to form a combined flat-field correction image.

The flat-field correction is applied to all readouts of the calibrated IR MultiAccum stack, as well as the single image produced by the CRCORR function.

All flat-field reference images are chosen from CDBS based on the DETECTOR, CCDAMP, and FILTER used for the observation. A sub-array science image uses the same reference file(s) as a full-size image; calwf3 extracts the appropriate region from the reference file(s) and applies it to the sub-array input image.
See the discussion of this step in ‘UVIS Flat-Field Image Correction’ for information regarding corrections for geometric distortion.


In [18]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':True,'nlin':True,'flat':True,'driz':None,'phot':None,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[7,:] = cube[:,jj,ii]
errout[7,:] = error[:,jj,ii]
dqout[7,:] = dq[:,jj,ii]
stepname.append('FLAT')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 66578.5234375   66525.1015625   66890.53125     31963.74414062
  34481.578125  ]
[ 319.56039429  319.37469482  320.55462646  196.53363037  209.08392334]
[ 2304.  2304.  2304.  2048.  2048.]


## Step 8: DRIZ – Combine the files in an association by calling [AstroDrizzle](http://drizzlepac.stsci.edu)


In [19]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':True,'nlin':True,'flat':True,'driz':True,'phot':None,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[8,:] = cube[:,jj,ii]
errout[8,:] = error[:,jj,ii]
dqout[8,:] = dq[:,jj,ii]
stepname.append('DRIZ')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 66578.5234375   66525.1015625   66890.53125     31963.74414062
  34481.578125  ]
[ 319.56039429  319.37469482  320.55462646  196.53363037  209.08392334]
[ 2304.  2304.  2304.  2048.  2048.]


## Step 9: PHOT – IR Photometry Keyword Calculation

- Header Switch: PHOTCORR

- Header Keywords Updated: PHOTMODE, PHOTFLAM, PHOTFNU, PHOTZPT, PHOTPLAM, PHOTBW

- Reference Files: GRAPHTAB (*_tmg.fits), COMPTAB (*_tmc.fits)

Before photometry can be derived from WFC3 observations, a transformation to absolute flux units must be done. calwf3 follows the WFPC2 and ACS methodology for calculating the photometry keywords in the calibration pipeline. The calibration reference file IMPHTTAB contains the values of the latest WFC3 photometry keywords as calculated by pysynphot as a function of wavelength for the various WFC3 detector and filter combinations.For further discussion of pysynphot, refer to Chapter 3 of the Introduction to the HST Data Handbooks.

During this process the keyword PHOTMODE is built to reflect the configuration of the instrument for the exposure (e.g., ‘WFC3,IR,F160W’). calwf3 then uses the PHOTMODE string with pysynphot to compute the total throughput for this instrument mode, based on the optics and filter throughputs and the detector QE. From that information, it computes values for the following photometry keywords.

PHOTFLAM: the inverse sensitivity in units of erg cm-2 A-1 electron-1

PHOTFNU: the inverse sensitivity in units of Jy sec-1 electron-1

PHOTZPT: the Space Telescope magnitude zero point

PHOTPLAM: the bandpass pivot wavelength

PHOTBW: the bandpass RMS width

Users who wish to convert calibrated IR images (which are in units of electrons per second) to flux units can simply multiply the image by the PHOTFNU keyword value.

In [20]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':True,'nlin':True,'flat':True,'driz':True,'phot':True,'unit':None,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)

hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[9,:] = cube[:,jj,ii]
errout[9,:] = error[:,jj,ii]
dqout[9,:] = dq[:,jj,ii]
stepname.append('PHOT')

calwf3 complete: output saved in icol26k3q_ima.fits
[ 66578.5234375   66525.1015625   66890.53125     31963.74414062
  34481.578125  ]
[ 319.56039429  319.37469482  320.55462646  196.53363037  209.08392334]
[ 2304.  2304.  2304.  2048.  2048.]


## Step 10: UNIT – IR Unit Conversion

- Header Switch: UNITCORR

- Header Keywords Updated: BUNIT

- Reference Files: None

This function simply converts the science data from a time-integrated signal to a signal rate, by dividing the science (SCI) and error (ERR) image arrays for each readout by the exposure time (TIME) image data. No reference file is needed.

Usually, the final units will be electrons per second, but if certain steps in the standard processing are omitted, the final units in the ima and flt files may be electrons, counts, or counts per second, where counts refers to the digitized signal from the FPA. The BUNIT keyword in the output files will always reflect the units of the data.`

In [21]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':True,'nlin':True,'flat':True,'driz':True,'phot':None,'unit':True,'cr':None}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[10,:] = cube[:,jj,ii]
errout[10,:] = error[:,jj,ii]
dqout[10,:] = dq[:,jj,ii]
stepname.append('UNIT')

calwf3 complete: output saved in icol26k3q_ima.fits
[   219.77955627    327.81820679    649.84790039  10900.60449219
  11842.19335938]
[  1.05488718   1.57379448   3.11421871  67.02391052  71.80682373]
[ 2304.  2304.  2304.  2048.  2048.]


## Step 11: CR – IR Up-The-Ramp Fitting and Cosmic-Ray Identification

- Header Switch: CRCORR

- Header Keywords Updated: None

- Reference Files: None

CRCORR combines the data from all readouts into a single image and in the process identifies and flags pixels suspected of containing cosmic-ray (CR) hits. The data from all readouts are analyzed pixel-by-pixel, iteratively computing a linear fit to the accumulating counts-versus-exposure time relation. Samples flagged as bad in the DQ arrays, such as when saturation occurs midway through the exposure, are rejected from the fitting process. CR hits are identified by searching for outliers from the fit results. The rejection threshold is set by the value in the ‘CRSIGMAS’ column of the Cosmic-Ray Rejection parameters reference table (CRREJTAB), which currently has a default value of 4s. When a CR hit is detected, a linear fit is then performed independently for the sets of readouts before and after the hit. Those fitting results are then again checked for outliers. This process is iterated until no new samples are rejected. Pixel samples identified as containing a CR hit are flagged in the DQ arrays of the intermediate MultiAccum (ima) file, with a DQ value of 8192. The pixel values in the SCI and ERR images of the ima file, however, are left unchanged. These DQ flags are only present in the ima file and do not get carried over into flt products which combine data from all readouts.

Once all outliers have been identified, a final count rate value, and its uncertainty, are determined for each pixel by computing the weighted mean of the slopes of each segment of non-flagged samples. The result of this operation is stored as a single imset in the output flt file. In the flt file the SCI array contains the final slope computed for each pixel, the ERR array contains the estimated uncertainty in the slope, the SAMP array contains the total number of non-flagged samples used to compute the slope, and the TIME array contains the total exposure time of those samples.
Pixels for which there are no unflagged samples, e.g., permanently hot or cold pixels, still get a slope computed, which is recorded in the SCI array of the output flt file, but they will also have their DQ flags recorded in the DQ array of the flt file. Users should therefore be careful to always check the flt file DQ arrays to help determine whether a given SCI image value is trustworthy for subsequent analysis.

The basic rule of thumb is that in order for a DQ value to propagate into the flt, it needs to be present in all the reads of the flt. The 8192 flag does not get propagated because it shows where during the ramp the cosmic ray showed up. Nominally, when looking at the flt file, calwf3 has already dealt with the cosmic ray and its effects are gone. If a user really wants to know about those cosmic rays, the ima files are available and contain the complete record of when and where exactly each cosmic ray showed up.

Another flag that is treated this way is saturation, flag value = 256. If a pixel is saturated in the last two reads of a ramp, then those two reads are flagged with 256 in the ima, calwf3 ignores those reads during line-fitting. The DQ value put into the flt file is 0 because calwf3 has already dealt with the saturation and the effects are not in the flt.

For pixels where calwf3 finds 4 or more cosmic rays up the ramp, it flags the pixel with flag value = 4 ("dead" or bad pixel), which does propagate to the flt. In that case, the thinking is that four large signal jumps for a given pixel in a single ramp means that something is going on with that pixel, and it should be thrown out.

In [22]:
os.remove(filebase+"_flt.fits")
os.remove(filebase+"_ima.fits")
stepdict = {'dqi':True,'zsig':True,'zoff':True,'dark':True,'blev':True,'nlin':True,'flat':True,'driz':True,'phot':True,'unit':True,'cr':True}
calwf3_run(file_raw, stepdict, kw=None, value=False)
hdulist=fits.open(filebase+'_ima.fits')
for i in range(1,25,5): 
    cube[int(i/5),:,:] = hdulist[i].data
    error[int(i/5),:,:] = hdulist[i+1].data
    dq[int(i/5),:,:] = hdulist[i+2].data
print(cube[:,jj,ii])
print(error[:,jj,ii])
print(dq[:,jj,ii])
listout[11,:] = cube[:,jj,ii]
errout[11,:] = error[:,jj,ii]
dqout[11,:] = dq[:,jj,ii]
stepname.append('CR')

calwf3 complete: output saved in icol26k3q_ima.fits
[   219.77955627    327.81820679    649.84790039  10900.60449219
  11842.19335938]
[  1.05488718   1.57379448   3.11421871  67.02391052  71.80682373]
[ 2304.  2304.  2304.  2048.  2048.]


## Done!

In [23]:
print(' ')
print('Here is the final summary for pixel',ii,jj)
np.set_printoptions(precision=3)
for i in range(12): 
    print(i,stepname[i],listout[i,:])
    print('   err',errout[i,:])
    print('    dq',dqout[i,:])
hdulist=fits.open(filebase+"_flt.fits")
countrate = hdulist[1].data
errorrate = hdulist[2].data
dqrate = hdulist[3].data
print('Here is the final countrate:',countrate[jj-5,ii-5])
print('        the final error    :',errorrate[jj-5,ii-5])
print('        the final dq       :',dqrate[jj-5,ii-5])


 
('Here is the final summary for pixel', 446, 721)
(0, 'RAW', array([ 54657.,  54631.,  54790.,  38868.,  25537.]))
('   err', array([ nan,  nan,  nan,  nan,  nan]))
('    dq', array([ nan,  nan,  nan,  nan,  nan]))
(1, 'DQI', array([ 54657.,  54631.,  54790.,  38868.,  25537.]))
('   err', array([ 153.076,  153.04 ,  153.261,  129.17 ,  104.823]))
('    dq', array([ 0.,  0.,  0.,  0.,  0.]))
(2, 'ZSIG', array([ 54657.,  54631.,  54790.,  38868.,  40529.]))
('   err', array([ 153.076,  153.04 ,  153.261,  129.17 ,  131.889]))
('    dq', array([    0.,     0.,     0.,     0.,  2048.]))
(3, 'ZOFF', array([ 29120.,  29094.,  29253.,  13331.,  14992.]))
('   err', array([ 111.888,  111.839,  112.142,   75.971,   80.507]))
('    dq', array([    0.,     0.,     0.,     0.,  2048.]))
(4, 'DARK', array([ 29115.756,  29093.289,  29254.418,  13331.677,  14992.   ]))
('   err', array([ 111.89 ,  111.841,  112.144,   75.974,   80.507]))
('    dq', array([    0.,     0.,     0.,     0.,  2048.]))


### Notes:

DQ = 2048 => Signal in Zero Read

DQ = 256  => Full Well saturation (2048+256=2304)